T mới làm có một đêm thôi lỗi nhiều lắm cứ góp ý

Link bài viết t tham khảo:

https://www.analyticsvidhya.com/blog/2014/11/text-data-cleaning-steps-python/

https://www.analyticsvidhya.com/blog/2020/04/beginners-guide-exploratory-data-analysis-text-data/

In [1]:
# import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math
import string

In [2]:
df = pd.read_csv('./UIT-ViSFD/Train.csv')

# Summary dữ liệu

Như tiêu đề thôi, t summary xíu

In [3]:
df.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


Do cần đánh giá cảm xúc bình luận nên ta chỉ cần 2 cột là comment và label (có thể cân nhắc cột n_star)

Tiến hành chọn cột cần cho đề bài

In [4]:
df = df[['comment', 'label']]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  7786 non-null   object
 1   label    7786 non-null   object
dtypes: object(2)
memory usage: 121.8+ KB


Chúng ta có 7786 comment với 7786 label tương ứng, nghĩa là t không cần xử lý dữ liệu rỗng ok nha mấy ô

# Lowercase các từ

Bước đầu t sẽ lowercase các từ để những công đoạn sau dễ xử lý hơn

In [6]:
df1 = df.copy()
df1.comment = df1.comment.apply(lambda x: x.lower())

In [7]:
df1.head()

,comment,label
0,mới mua máy này tại thegioididong thốt nốt cảm...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,{FEATURES#Negative};
3,"mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,"mới mua sài được 1 tháng thấy pin rất trâu, sà...",{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# Xử lý stop-word

Stop-word là các từ thường không có tác dụng trong câu

Thí dụ như từ "mình" đi, "mình thấy điện thoại khá đẹp" -> "thấy điện thoại khá đẹp"

Nội dung câu vẫn không thay đổi, đánh giá rằng điện thoại có ngoại hình đẹp

Stop-word trong thư viện nltk không có ngôn ngữ Tiếng Việt, nên t sẽ sử dụng một repository trên GitHub

Link repository GitHub: https://github.com/stopwords/vietnamese-stopwords/tree/master

In [8]:
# Vì comment không có dash nên sử dụng bản không có dash
stop_words = pd.read_csv(rf'vietnamese-stopwords-master\vietnamese-stopwords.txt', header=None)

In [9]:
stop_words.head()

,0
0,a lô
1,a ha
2,ai
3,ai ai
4,ai nấy


In [10]:
stop_words = stop_words[0].tolist()

In [11]:
# Bỏ đi các từ stop word
df2 = df1.copy()
df2['comment'] = df2['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

Check con hàng

In [12]:
df1.head()

,comment,label
0,mới mua máy này tại thegioididong thốt nốt cảm...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,{FEATURES#Negative};
3,"mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,"mới mua sài được 1 tháng thấy pin rất trâu, sà...",{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [13]:
df2.head()

,comment,label
0,mua máy thegioididong nốt cảm ok bin trâu chụp...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém miễn chê mua 8/3/2019 tình trạng pin 88%,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,gọi điện thoại màn hình chấm nháy camera vậy(l...,{FEATURES#Negative};
3,"cập nhật mềm , bớt tốn pin, thử rồi, ok, vân k...",{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,"mua sài 1 pin trâu, sài bao mượt . 1 lỗi nhạc ...",{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


Mấy ô có thể check 2 cái và thấy được sự khác biết liền :smile:

# Loại bỏ dấu câu và emoji

Dấu câu chỉ có tác dụng trong văn học, ta hoàn toàn có thể nghỉ chơi với nó

Emoji thì thường không thể hiện được ý nghĩa câu nên bỏ luôn

In [14]:
df3 = df2.copy()
df3['comment'] = df3['comment'].str.replace(r'[^\w\s]', '', regex=True)

In [15]:
df2.head()

,comment,label
0,mua máy thegioididong nốt cảm ok bin trâu chụp...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém miễn chê mua 8/3/2019 tình trạng pin 88%,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,gọi điện thoại màn hình chấm nháy camera vậy(l...,{FEATURES#Negative};
3,"cập nhật mềm , bớt tốn pin, thử rồi, ok, vân k...",{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,"mua sài 1 pin trâu, sài bao mượt . 1 lỗi nhạc ...",{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [16]:
df3.head()

,comment,label
0,mua máy thegioididong nốt cảm ok bin trâu chụp...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém miễn chê mua 832019 tình trạng pin 88,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,gọi điện thoại màn hình chấm nháy camera vậylú...,{FEATURES#Negative};
3,cập nhật mềm bớt tốn pin thử rồi ok vân ko nhạy,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,mua sài 1 pin trâu sài bao mượt 1 lỗi nhạc ta...,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


Đây là 1 câu có emoji mà t kiếm được

In [17]:
df2["comment"][97]

'1. máy game, lướt web..., sử dụng pin 1 16 tiếng ( ngủ ngủ chẳng động ) . 2 mn 2. nhạc cà giật cà giật 3. video mờ chán okie 🙆'

In [18]:
df3["comment"][97]

'1 máy game lướt web sử dụng pin 1 16 tiếng  ngủ ngủ chẳng động   2 mn 2 nhạc cà giật cà giật 3 video mờ chán okie '

Dấu câu và emoji đi rồi ha :blush:

In [19]:
# df3["comment"].to_csv('sub3.csv', index=False)

T nhận ra là bản stop-word t đang xài không có từ chửi thề mấy ô ạ.

Ví dụ như "mượt vl" thì nó vẫn cứ là "mượt vl" thôi

Thế nên t làm tạm một list chửi thề để remove

In [20]:
bad_words = [['loz', 'dm', 'vl', 'đéo', 'đcmn', 'qq', 'vcl', 'vãi', 'chưởng', 'vc', 'thề', 't']]

Note: ở một vài comment 'vc' là 'việc'

In [21]:
# Bỏ đi các từ chửi thề
df4 = df3.copy()
df4['comment'] = df4['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in bad_words]))

In [22]:
# df4.to_csv("sub4.csv")